## Import Data

In [2]:
import os
import numpy as np
import pandas as pd
import csv
import sys
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
import joblib
import optuna

from rdkit import Chem
from rdkit.Chem import AllChem, MolFromSmiles
from rdkit.Chem import rdMolDescriptors as rd
import torch as th
import torch.nn.functional as fn

In [4]:
# df = pd.read_csv('data/smiles_embeddings_all.csv')
# df.shape

(8807, 403)

In [4]:
# # Drop the untitled column
# df.drop(['Unnamed: 0'], inplace=True, axis=1)
# df.head()

,DrugBankID,SMILES,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_390,embedding_391,embedding_392,embedding_393,embedding_394,embedding_395,embedding_396,embedding_397,embedding_398,embedding_399
0,Compound::DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,-0.659347,-0.344230,-0.578348,-0.670077,0.073021,-0.844866,-0.529099,-0.797582,...,0.722160,-0.361777,-0.069529,0.719951,0.765324,-0.828312,-0.750704,-0.360401,-0.359896,-0.820253
1,Compound::DB00007,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,-0.606968,-0.781302,-0.730112,-0.868258,0.363371,-0.388290,0.217138,-0.640213,...,-0.355017,0.657896,0.039386,-0.395858,-0.218164,-0.540272,-0.603087,-0.853275,0.545669,-0.836144
2,Compound::DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,-0.470256,-0.885203,-0.623956,-0.507102,0.395201,-0.298908,-0.156826,-0.697836,...,-0.430036,0.563329,0.419391,-0.074537,0.266462,-0.631508,-0.637233,-0.816603,0.416263,-0.720862
3,Compound::DB00027,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,-0.761337,-0.709398,0.665223,-0.388625,0.218644,-0.416196,0.594226,-0.497919,...,-0.721291,0.630432,0.817134,0.515158,-0.247925,-0.523338,-0.827212,-0.751571,-0.640211,-0.756264
4,Compound::DB00035,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,-0.764784,-0.956786,-0.445434,-0.611626,0.335249,0.044306,-0.635370,-0.706563,...,-0.204678,-0.404126,-0.018485,-0.401169,0.025806,-0.664728,-0.856459,-0.417736,-0.386456,-0.626406


## Split data into train and test set 
Store the data as a CSV file.

In [5]:
# Split the DataFrame into train and test sets (80% train, 20% test)
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# # Save the train and test sets to CSV files
# train_df.to_csv('data/train.csv', index=False)
# test_df.to_csv('data/test.csv', index=False)

## Morgan Fingerprint Count Model

In [5]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [6]:
RADIUS = 2
N_BITS = 2048

# Function to calculate Morgan fingerprint count
def calculate_morgan_fingerprint_count(smiles, radius=RADIUS, n_bits=N_BITS):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        fingerprint = rd.GetHashedMorganFingerprint(mol, radius=radius, nBits=n_bits)
        arr = np.zeros((n_bits,), dtype=np.uint8)
        for idx, count in fingerprint.GetNonzeroElements().items():
            arr[idx] = count if count < 255 else 255
        return np.array(arr, dtype=np.uint8)
    else:
        return None

# Function to preprocess data and create embeddings
def preprocess_data(df):
    # Get the target embeddings
    embeddings = df.iloc[:, 2:].values
    # create fingerprint column
    df['morgan_fingerprint_count'] = df['SMILES'].apply(calculate_morgan_fingerprint_count)
    df = df.dropna()
    # Extract the fingerprints as a NumPy array
    morgan_fingerprints = np.array(df['morgan_fingerprint_count'].tolist())
    return morgan_fingerprints, embeddings

In [7]:
# The morgan_fingerprints is X, our features
# The embeddings is y, our target variable
X_train, y_train = preprocess_data(train_df)
X_test, y_test = preprocess_data(test_df)

print("The length of X_train is:", len(X_train))
print("The length of X_test is:", len(X_test))

[18:21:56] Unusual charge on atom 0 number of radical electrons set to zero


The length of X_train is: 7045
The length of X_test is: 1762


In [8]:
print(X_train.shape)
print(X_train)

(7045, 2048)
[[0 5 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]


## Train Model with XGBoost

In [9]:
def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "verbosity": 0,
        "n_estimators": trial.suggest_int("n_estimators", 100, 300),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        "n_jobs": -1,  # Set to -1 to use all available CPU cores
        "tree_method": "hist",  # Enable GPU acceleration
        "device": "cuda",  # Specify GPU as the device
    }

    # Create an XGBoost regressor with the suggested hyperparameters
    xgb_regressor = xgb.XGBRegressor(**params)
    xgb_regressor.fit(X_train, y_train)
    # Make predictions on the validation set
    y_pred = xgb_regressor.predict(X_test)
    # Calculate mean squared error for each target variable
    mse_per_target = np.mean((y_test - y_pred)**2, axis=0)
    print("mse per target", mse_per_target)
    # Average the mean squared errors across all target variables
    mse = np.mean(mse_per_target)
    return mse

In [ ]:
# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

# Print the best hyperparameters
print('Best trial:')
trial = study.best_trial
print(f'Params: {trial.params}')
print(f'Mean Squared Error: {trial.value}')

[I 2023-12-31 18:23:28,495] A new study created in memory with name: no-name-161b46cf-2211-4ed6-98af-6d3f2d8b105c
[I 2023-12-31 19:02:10,283] Trial 0 finished with value: 0.2234319876547247 and parameters: {'n_estimators': 137, 'learning_rate': 0.08505415703062957, 'max_depth': 9, 'subsample': 0.23700366921758392, 'colsample_bytree': 0.3684126659150004, 'min_child_weight': 6}. Best is trial 0 with value: 0.2234319876547247.


mse per target [0.24401477 0.21988327 0.22308218 0.23034952 0.13363272 0.2070944
 0.16067726 0.24104865 0.23062313 0.22809819 0.21611139 0.23648347
 0.22092155 0.23497385 0.24713861 0.21378004 0.18803587 0.23956591
 0.23981751 0.19940742 0.15140558 0.22013295 0.23594081 0.21757178
 0.24929153 0.24819599 0.25332411 0.23714813 0.2248641  0.25340141
 0.23936392 0.25700826 0.23498098 0.22320984 0.2440283  0.24395086
 0.2032307  0.24244156 0.26234791 0.22028927 0.24247963 0.19166008
 0.22684375 0.14792878 0.19803089 0.1692286  0.1914409  0.20440113
 0.2183088  0.22706338 0.22457075 0.22267683 0.26044647 0.22368199
 0.2065936  0.24208549 0.20854059 0.23411859 0.21429483 0.22696256
 0.19658163 0.23646185 0.25905152 0.19392108 0.23649673 0.24018847
 0.26565221 0.24970262 0.22018264 0.23560025 0.23596591 0.27107562
 0.19205248 0.21723801 0.20534713 0.22701642 0.17820268 0.23283847
 0.24262051 0.21489295 0.24122132 0.22963617 0.23023323 0.22875557
 0.2070426  0.24364424 0.26720094 0.25123693 0.1

[I 2023-12-31 19:35:21,382] Trial 1 finished with value: 0.2182944925633194 and parameters: {'n_estimators': 100, 'learning_rate': 0.06385350022045472, 'max_depth': 8, 'subsample': 0.9317419548487089, 'colsample_bytree': 0.4485022846375093, 'min_child_weight': 2}. Best is trial 1 with value: 0.2182944925633194.


mse per target [0.23777788 0.21504428 0.21550841 0.22501811 0.12653626 0.20272544
 0.15685541 0.23777368 0.22631552 0.21976389 0.20898294 0.23094522
 0.21368052 0.22989416 0.24160203 0.2044308  0.18273846 0.23474648
 0.23329787 0.19838279 0.14819713 0.21751696 0.23149844 0.21539175
 0.24541135 0.23874834 0.24798754 0.22905532 0.21744373 0.24020099
 0.23577338 0.24485907 0.22879114 0.21748614 0.23992553 0.2388461
 0.20173302 0.23888338 0.25592324 0.2147361  0.23986609 0.18449361
 0.21858037 0.14153979 0.19312317 0.1655591  0.19208441 0.19948494
 0.21233018 0.22100028 0.22091539 0.22091232 0.25164952 0.21597188
 0.20077452 0.23745927 0.20345098 0.22943236 0.21159964 0.22618206
 0.18812512 0.23242554 0.25570591 0.18586707 0.23062178 0.23834892
 0.25692132 0.24603425 0.21996729 0.22742213 0.23288855 0.25982428
 0.1870297  0.21130993 0.2016091  0.21823379 0.17756104 0.22491516
 0.23925473 0.20805273 0.23114361 0.22423568 0.22532362 0.2278547
 0.20429103 0.24029717 0.25883547 0.24069157 0.17

[I 2024-01-01 03:00:21,880] Trial 2 finished with value: 0.23685215237872953 and parameters: {'n_estimators': 213, 'learning_rate': 0.0023729179623852466, 'max_depth': 8, 'subsample': 0.5123165942052653, 'colsample_bytree': 0.2915284167050504, 'min_child_weight': 9}. Best is trial 1 with value: 0.2182944925633194.


mse per target [0.25439806 0.22734956 0.22744598 0.25209749 0.13186153 0.25066521
 0.16890267 0.2737274  0.24016259 0.25072029 0.22268951 0.27701596
 0.23070539 0.24770093 0.249762   0.21559281 0.19750578 0.27925927
 0.2726033  0.25480391 0.1639254  0.23585419 0.24158998 0.22522357
 0.25506048 0.25123064 0.26673954 0.25194508 0.2350422  0.2660232
 0.25503092 0.26243806 0.27511777 0.23226549 0.25974139 0.26571237
 0.24414242 0.25392568 0.26593627 0.23070363 0.25684373 0.22108797
 0.23386703 0.1431722  0.20872968 0.19023215 0.23187649 0.22352565
 0.25584519 0.23624586 0.25243655 0.2529794  0.2652254  0.22458276
 0.22112024 0.2615448  0.21728414 0.24546301 0.22755064 0.23155619
 0.1924558  0.2521477  0.26948401 0.19879074 0.24065862 0.24810294
 0.27572606 0.2706145  0.22578631 0.25600016 0.24633964 0.27942067
 0.20283878 0.22025749 0.21347004 0.25217124 0.19309482 0.23469439
 0.25711259 0.22126869 0.25445951 0.24988034 0.24771491 0.24568588
 0.21642697 0.25430341 0.26947741 0.25078068 0.1

In [ ]:
# Extract the best hyperparameters
best_params = study.best_trial.params
# Create an XGBoost regressor with the best hyperparameters
best_xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
# Train the final model using the full training set
best_xgb_regressor.fit(X_train, y_train)
# Make predictions on your test set or perform any further evaluation
y_test_pred = best_xgb_regressor.predict(X_test)

In [ ]:
# Save the trained model to a file
model_filename = 'xgboost_mfc_model.bin'
joblib.dump(best_xgb_regressor, model_filename)

In [ ]:
# Load the saved model
model = joblib.load('xgboost_mfc_model.bin')

# Make predictions on the test set
mfc_embeddings = model.predict(X_test)
train_mfc_embeddings = model.predict(X_train)
print(mfc_embeddings)
print(train_mfc_embeddings)

In [ ]:
# Calculate R-squared score for each output
r2_scores_test = [r2_score(y_test[:, i], mfc_embeddings[:, i]) for i in range(y_test.shape[1])]
r2_scores_train = [r2_score(y_train[:, i], train_mfc_embeddings[:, i]) for i in range(y_train.shape[1])]

# Print the average R-squared score
average_r2_test = np.mean(r2_scores_test)
average_r2_train = np.mean(r2_scores_train)

print(f'Average Train R-squared Score: {average_r2_train:.4f}')
print(f'Average Test R-squared Score: {average_r2_test:.4f}')

## Evaluate Embeddings

### Column-wise evaluation
Compare the embeddings based on the index (original and predicted) of all the drugs


In [ ]:
def col_comparison(embedding_index, original_embeddings=y_test, predicted_embeddings=mfc_embeddings):
    '''Function to compare the original embeddings with the predicted embeddings.
    It uses a regression plot and Pearson correlation for comparison.
        
    Parameter
    ----------
    embedding_index (int): number specifying the embedding you want to compare. It ranges from 0-399.
    original_embeddings (np.array): The original embeddings from the knowledge graph.
    predicted_embeddingns (np.array): The embeddings predicted by the model
    
    Returns
    -------
    pearson_corr (np.array): person correlation score.
    '''
    # Extract embedding_index for both arrays
    embedding_index_y_test = original_embeddings[:, embedding_index]
    embedding_index_mf = predicted_embeddings[:, embedding_index]

    # Create a DataFrame for Seaborn plotting
    emb_df = pd.DataFrame({'Embedding_index_y_test': embedding_index_y_test,
                       'Embedding_index_mf': embedding_index_mf})

    # Plot the regression plot with Pearson correlation
    plt.figure(figsize=(10, 6))
    sns.regplot(x='Embedding_index_y_test', y='Embedding_index_mf', data=emb_df, scatter_kws={'s': 10})
    plt.title(f'Regression Plot of Embedding_{embedding_index}  for y_test and mf_embeddings')
    plt.xlabel(f'Embedding_{embedding_index} in y_test')
    plt.ylabel(f'Embedding_{embedding_index} in mf_embeddings')
    plt.show()

    # Calculate the Pearson correlation
    pearson_corr = np.corrcoef(embedding_index_y_test, embedding_index_mf)[0, 1]
    print(f'Pearson Correlation: {pearson_corr:.4f}')

#### Test Set

In [ ]:
col_comparison(0)

In [ ]:
col_comparison(1)

In [ ]:
col_comparison(2)

#### Train Set

In [ ]:
col_comparison(0, 
               original_embeddings=y_train, 
               predicted_embeddings=train_mfc_embeddings)

In [ ]:
col_comparison(1, 
               original_embeddings=y_train, 
               predicted_embeddings=train_mfc_embeddings)

### Row-wise Comparison of Embeddings
Compare the 400 embeddings (original and predicted) of a particular drug

In [ ]:
def row_comparison(drug_index, original_embeddings=y_test, predicted_embeddings=mfc_embeddings):
    '''Function to compare the 400 original embeddings with the predicted embeddings for a drug
    It uses a regression plot and Pearson correlation for comparison.
        
    Parameter
    ----------
    drug_index (int): number specifying the drug embedding you want to compare. It ranges from 0-1761.
    original_embeddings (np.array): The original embeddings from the knowledge graph.
    predicted_embeddingns (np.array): The embeddings predicted by the model
    
    Returns
    -------
    pearson_corr (np.array): person correlation score.
    '''
    # Extract embeddings for the specified drug
    embeddings_y_test = original_embeddings[drug_index]
    embeddings_mf = predicted_embeddings[drug_index]

    # Create a DataFrame for Seaborn plotting
    df = pd.DataFrame({'Embeddings_y_test': embeddings_y_test,
                       'Embeddings_mf': embeddings_mf})

    # Plot the regression plot
    plt.figure(figsize=(10, 6))
    sns.regplot(x='Embeddings_y_test', y='Embeddings_mf', data=df, scatter_kws={'s': 10})
    plt.title(f'Regression Plot of Embeddings for Drug {drug_index}')
    plt.xlabel(f'Original Embeddings for Drug {drug_index}')
    plt.ylabel(f'Predicted Embeddings for Drug {drug_index}')
    plt.show()

     # Calculate the Pearson correlation
    pearson_corr = np.corrcoef(embeddings_y_test, embeddings_mf)[0, 1]
    print(f'Pearson Correlation: {pearson_corr:.4f}')

#### Test Set

In [ ]:
row_comparison(0)

In [ ]:
row_comparison(1)

#### Train Set

In [ ]:
row_comparison(0, 
               original_embeddings=y_train, 
               predicted_embeddings=train_mfc_embeddings)

In [ ]:
row_comparison(1, 
               original_embeddings=y_train, 
               predicted_embeddings=train_mfc_embeddings)

## Predict the Edge Score

In [ ]:
COV_disease_list = [
'Disease::SARS-CoV2 E',
'Disease::SARS-CoV2 M',
'Disease::SARS-CoV2 N',
'Disease::SARS-CoV2 Spike',
'Disease::SARS-CoV2 nsp1',
'Disease::SARS-CoV2 nsp10',
'Disease::SARS-CoV2 nsp11',
'Disease::SARS-CoV2 nsp12',
'Disease::SARS-CoV2 nsp13',
'Disease::SARS-CoV2 nsp14',
'Disease::SARS-CoV2 nsp15',
'Disease::SARS-CoV2 nsp2',
'Disease::SARS-CoV2 nsp4',
'Disease::SARS-CoV2 nsp5',
'Disease::SARS-CoV2 nsp5_C145A',
'Disease::SARS-CoV2 nsp6',
'Disease::SARS-CoV2 nsp7',
'Disease::SARS-CoV2 nsp8',
'Disease::SARS-CoV2 nsp9',
'Disease::SARS-CoV2 orf10',
'Disease::SARS-CoV2 orf3a',
'Disease::SARS-CoV2 orf3b',
'Disease::SARS-CoV2 orf6',
'Disease::SARS-CoV2 orf7a',
'Disease::SARS-CoV2 orf8',
'Disease::SARS-CoV2 orf9b',
'Disease::SARS-CoV2 orf9c',
'Disease::MESH:D045169',
'Disease::MESH:D045473',
'Disease::MESH:D001351',
'Disease::MESH:D065207',
'Disease::MESH:D028941',
'Disease::MESH:D058957',
'Disease::MESH:D006517'
]

treatment = ['Hetionet::CtD::Compound:Disease','GNBR::T::Compound:Disease']


gamma = 12.0

def transE_l2(head, rel, tail):
    score = head + rel - tail
    return gamma - th.norm(score, p=2, dim=-1)


def edge_score(embeddings):
    '''Function to calculate the edge scores.

    Argument
    ---------
    embeddings (array). Array of size 400 containing 
            the embeddings of the SMILES molecule.

    Returns
    --------
    scores (tensor). Tensor showing the edge score for 
            each disease based on the drug_embeddings, relation_embeddings,
            and COVID_disease embeddings.
    '''
    
    # Load entity and relation mapping files
    entity_idmap_file = 'data/entities.tsv'
    relation_idmap_file = 'data/relations.tsv'

    # Get drugname/disease name to entity ID mappings
    entity_map = {}
    entity_id_map = {}
    relation_map = {}
    
    with open(entity_idmap_file, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name', 'id'])
        for row_val in reader:
            entity_map[row_val['name']] = int(row_val['id'])
            entity_id_map[int(row_val['id'])] = row_val['name']

    with open(relation_idmap_file, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter='\t', fieldnames=['name', 'id'])
        for row_val in reader:
            relation_map[row_val['name']] = int(row_val['id'])

    # Handle the ID mapping
    # drug_ids = [entity_map[drug] for drug in drug_list]
    disease_ids = [entity_map[disease] for disease in COV_disease_list]
    treatment_rid = [relation_map[treat] for treat in treatment]

    # Load embeddings
    entity_emb = np.load('data/DRKG_TransE_l2_entity.npy')
    rel_emb = np.load('data/DRKG_TransE_l2_relation.npy')

    # drug_ids = th.tensor(drug_ids).long()
    disease_ids = th.tensor(disease_ids).long()
    treatment_rid = th.tensor(treatment_rid)

    # Use our model's embeddings here
    drug_emb = th.tensor(embeddings)
    # drug_emb = th.tensor(entity_emb[drug_ids]) # get embeddings from knowledge graph
    
    treatment_embs = [th.tensor(rel_emb[rid]) for rid in treatment_rid]

    scores_per_disease = []
    for rid in range(len(treatment_embs)):
        treatment_emb=treatment_embs[rid]
        for disease_id in disease_ids:
            disease_emb = entity_emb[disease_id]
            score = fn.logsigmoid(transE_l2(drug_emb, treatment_emb, disease_emb))
            scores_per_disease.append(score)
            
    # Convert scores_per_disease to a list of tensors
    scores_tensors = [th.tensor(scores, dtype=th.float32) for scores in scores_per_disease]
    # Get the first combo edge score
    return scores_tensors[0]

    # # Ensure the list of tensors is not empty before calculating the max
    # if scores_tensors:
    #     # Stack the list of tensors along a new dimension (axis 0)
    #     scores_tensor = th.stack(scores_tensors, dim=0)
    #     # Calculate the maximum score for each drug along the existing dimension (axis 0)
    #     max_scores, _ = th.max(scores_tensor, dim=0)
    #     # Print the shape and content of the max_scores tensor
    #     print("Shape of max_scores:", max_scores.shape)
    #     # print("Maximum scores for each drug:", max_scores.tolist())  # Use .tolist() to get a Python list
    #     return max_scores
    # else:
    #     print("The list of tensors is empty.")
    #     return None

In [ ]:
# y_test has the original embeddings
# Predict the original_edge_score
original_edge_score = edge_score(y_test)

# Predict the Morgan Fingerprint Count edge score
mfc_edge_score = edge_score(mfc_embeddings)

In [ ]:
mfc_edge_score

In [ ]:
original_edge_score

## Plot result

In [ ]:
# Calculate R-squared score
r2 = r2_score(original_edge_score, mfc_edge_score)

# Calculate Mean Squared Error
mse = mean_squared_error(original_edge_score, mfc_edge_score)

# Plot the line graph
plt.figure(figsize=(10, 6))
plt.plot(original_edge_score[:50], label='Original Edge Score', marker='o')
plt.plot(mfc_edge_score[:50], label='Morgan Fingerprint Count Edge Score', marker='o')
plt.title('Comparison of Original and Morgan Fingerprint Count Edge Scores')
plt.xlabel('Data Points')
plt.ylabel('Edge Score')
plt.legend()
plt.show()

# Print R-squared and MSE
print(f'R-squared Score: {r2:.4f}')
print(f'Mean Squared Error: {mse:.4f}')

In [ ]:
# Plot the scatter plot using Matplotlib
plt.figure(figsize=(10, 6))
plt.scatter(x=original_edge_score, y=mfc_edge_score)
plt.title('Scatter Plot of Real vs. Predicted Edge Scores')
plt.xlabel('Real Edge Score')
plt.ylabel('Predicted Edge Score')
plt.show()